In [182]:
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib

In [183]:
categories = ["temperature", "specific_humidity", "u_component_of_wind", "v_component_of_wind", "geopotential"]
categories_deu = ["Temperatur", "Spezifische Feuchte", "U Windkomponente", "V Windkomponente", "Geopotential"]

models = ["IFS Hres", "Graphcast Operational", "Pangu-Weather Operational", "Unser FuXi"]
day_slices = [1, 3, 5, 7, 9]
forecast_days = [f"Tag {i}" for i in day_slices]
x = np.arange(len(forecast_days))
cmap = matplotlib.colormaps['viridis']
colors = cmap(np.linspace(0, 1, 4))

In [184]:
graphcast = xr.open_dataset("../data/eval/graphcast_operational.nc").isel(lead_time=[i for i in range(39)])
ifs = xr.open_dataset("../data/eval/ifs_hres.nc").isel(lead_time=[i for i in range(39)])
pangu = xr.open_dataset("../data/eval/pangu_operational.nc").isel(lead_time=[i for i in range(39)])
fuxi = xr.open_dataset("../data/eval/our_fuxi.nc").isel(lead_time=[i for i in range(1, 40)])

In [174]:
x_label = [i for i in range(39)]
x_plot = [f"Tag {int(np.timedelta64((i+1)*6, 'h')/ np.timedelta64(1, 'D'))}" if np.timedelta64((i+1)*6, 'h')/ np.timedelta64(1, 'D') % 1 == 0 else "" for i in x_label]

fig = plt.figure(figsize=(14,8))
ax = plt.subplot(111)

for variable in ["temperature", "specific_humidity", "u_component_of_wind", "v_component_of_wind", "geopotential"]:
    y = np.array(fuxi[variable].sel(metric='acc', region='global')).flatten()
    ax.plot(x_label,y, label=variable)

ax.set_xticks(x_label)
ax.set_xticklabels(x_plot)
plt.ylabel("ACC")

ax.legend(title='Variable', prop={'size': 20})

plt.savefig('../figures/acc.png')
plt.close()

In [187]:
x_label = [i for i in range(39)]
x_plot = [f"Tag {int(np.timedelta64((i+1)*6, 'h')/ np.timedelta64(1, 'D'))}" if np.timedelta64((i+1)*6, 'h')/ np.timedelta64(1, 'D') % 1 == 0 else "" for i in x_label]

for variable in ["temperature", "specific_humidity", "u_component_of_wind", "v_component_of_wind", "geopotential"]:
    y = np.sqrt(np.array(fuxi[variable].sel(metric='mse', region='global')).flatten())
    fig = plt.figure(figsize=(14,8))
    ax = plt.subplot(111)
    ax.plot(x_label,y, label=variable)

    ax.set_xticks(x_label)
    ax.set_xticklabels(x_plot, fontsize=15)
    ax.yaxis.label.set_size(15)
    plt.ylabel("RMSE", fontsize=15)
    plt.savefig(f"../figures/rmse_{variable}.png")
    # plt.show()
    plt.close()

In [158]:
slices = [[0, 1, 2],
          [3, 4, 5, 6], [7, 8, 9, 10], [11, 12, 13, 14], [15, 16, 17, 18], [19, 20, 21, 22],
          [23, 24, 25, 26], [27, 28, 29, 30], [31, 32, 33, 34], [35, 36, 37, 38]]
for idx, cat in enumerate(categories):
    gcv_val = np.array(graphcast[cat].sel(region='global', level=850, metric='acc'))
    ifs_vals = np.array(ifs[cat].sel(region='global', level=850, metric='acc'))
    pangu_vals = np.array(pangu[cat].sel(region='global', level=850, metric='acc'))
    fuxi_vals = np.array(fuxi[cat].sel(region='global', level=850, metric='acc'))
    gcv_days = []
    ifs_days = []
    pangu_days = []
    fuxi_days = []
    for sl in slices:
        gcv_days.append(gcv_val.take(sl).mean())
        ifs_days.append(ifs_vals.take(sl).mean())
        pangu_days.append(pangu_vals.take(sl).mean())
        fuxi_days.append(fuxi_vals.take(sl).mean())
    gcv_days = np.array(gcv_days)
    ifs_days = np.array(ifs_days)
    pangu_days = np.array(pangu_days)
    fuxi_days = np.array(fuxi_days)

    fig, ax = plt.subplots(figsize=(14, 8))
    plt.bar(x - 0.4, gcv_days.take(day_slices), width=0.2, label='Graphcast', color=colors[0])
    plt.bar(x - 0.2, ifs_days.take(day_slices), width=0.2, label='IFS HRes', color=colors[1])
    plt.bar(x, pangu_days.take(day_slices), width=0.2, label='Pangu-Weather', color=colors[2])
    plt.bar(x + 0.2, fuxi_days.take(day_slices), width=0.2, label='Unser FuXi', color=colors[3])

    ax.set_xticks(x)
    ax.set_xticklabels(forecast_days)

    plt.ylabel('ACC')
    plt.xticks(rotation=45)
    plt.legend(title='Modell', prop={'size': 20})
    plt.tight_layout()
    # Show plot
    plt.savefig(f"../figures/modellvergleich_{cat}_acc.png")
    plt.close()

In [159]:
slices = [[0, 1, 2],
          [3, 4, 5, 6], [7, 8, 9, 10], [11, 12, 13, 14], [15, 16, 17, 18], [19, 20, 21, 22],
          [23, 24, 25, 26], [27, 28, 29, 30], [31, 32, 33, 34], [35, 36, 37, 38]]
for idx, cat in enumerate(categories):
    gcv_val = np.sqrt(np.array(graphcast[cat].sel(region='global', level=850, metric='mse')))
    ifs_vals = np.sqrt(np.array(ifs[cat].sel(region='global', level=850, metric='mse')))
    pangu_vals = np.sqrt(np.array(pangu[cat].sel(region='global', level=850, metric='mse')))
    fuxi_vals = np.sqrt(np.array(fuxi[cat].sel(region='global', level=850, metric='mse')))
    gcv_days = []
    ifs_days = []
    pangu_days = []
    fuxi_days = []
    for sl in slices:
        gcv_days.append(gcv_val.take(sl).mean())
        ifs_days.append(ifs_vals.take(sl).mean())
        pangu_days.append(pangu_vals.take(sl).mean())
        fuxi_days.append(fuxi_vals.take(sl).mean())
    gcv_days = np.array(gcv_days)
    ifs_days = np.array(ifs_days)
    pangu_days = np.array(pangu_days)
    fuxi_days = np.array(fuxi_days)

    fig, ax = plt.subplots(figsize=(14, 8))
    plt.bar(x - 0.4, gcv_days.take(day_slices), width=0.2, label='Graphcast', color=colors[0])
    plt.bar(x - 0.2, ifs_days.take(day_slices), width=0.2, label='IFS HRes', color=colors[1])
    plt.bar(x, pangu_days.take(day_slices), width=0.2, label='Pangu-Weather', color=colors[2])
    plt.bar(x + 0.2, fuxi_days.take(day_slices), width=0.2, label='Unser FuXi', color=colors[3])

    ax.set_xticks(x)
    ax.set_xticklabels(forecast_days)

    plt.ylabel('RMSE')
    plt.xticks(rotation=45)
    plt.legend(title='Modell', prop={'size': 20})
    plt.tight_layout()
    # Show plot
    plt.savefig(f"../figures/modellvergleich_{cat}_rmse.png")
    plt.close()

In [160]:
slices = [[0, 1, 2],
          [3, 4, 5, 6], [7, 8, 9, 10], [11, 12, 13, 14], [15, 16, 17, 18], [19, 20, 21, 22],
          [23, 24, 25, 26], [27, 28, 29, 30], [31, 32, 33, 34], [35, 36, 37, 38]]
x = [i for i in range(39)]
x_ticks = [f"Tag {int(np.timedelta64((i+1)*6, 'h')/ np.timedelta64(1, 'D'))}" if np.timedelta64((i+1)*6, 'h')/ np.timedelta64(1, 'D') % 1 == 0 else "" for i in x]
gcv_days_com = []
ifs_days_com = []
pangu_days_com = []
fuxi_days_com = []
for idx, cat in enumerate(categories):
    gcv_val = np.array(graphcast[cat].sel(region='global', level=850, metric='acc'))
    ifs_vals = np.array(ifs[cat].sel(region='global', level=850, metric='acc'))
    pangu_vals = np.array(pangu[cat].sel(region='global', level=850, metric='acc'))
    fuxi_vals = np.array(fuxi[cat].sel(region='global', level=850, metric='acc'))
    gcv_days_com.append(gcv_val)
    ifs_days_com.append(ifs_vals)
    pangu_days_com.append(pangu_vals)
    fuxi_days_com.append(fuxi_vals)

gcv_days_com = np.array(gcv_days_com).mean(axis=0)
ifs_days_com = np.array(ifs_days_com).mean(axis=0)
pangu_days_com = np.array(pangu_days_com).mean(axis=0)
fuxi_days_com = np.array(fuxi_days_com).mean(axis=0)

fig, ax = plt.subplots(figsize=(14, 8))
plt.plot(x, gcv_days_com, label='Graphcast', color=colors[0])
plt.plot(x, ifs_days_com, label='IFS HRes', color=colors[1])
plt.plot(x, pangu_days_com, label='Pangu-Weather', color=colors[2])
plt.plot(x, fuxi_days_com, label='Unser FuXi', color=colors[3])

ax.set_xticks(x)
ax.set_xticklabels(x_ticks)

plt.ylabel('ACC')
plt.xlabel('Kategorie und Tag der Vorhersage')
plt.xticks(rotation=45)
plt.legend(title='Modell', prop={'size': 20})
plt.tight_layout()
# Show plot
plt.savefig(f"../figures/modellvergleich_cat_mean_acc.png")
plt.close()